In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier

def selectkbest(indep_X, dep_Y, n):
    test = SelectKBest(score_func=chi2, k=n)
    fit1 = test.fit(indep_X, dep_Y)
    selectk_features = fit1.transform(indep_X)
    selected_feature_names = fit1.get_feature_names_out(input_features=indep_X.columns)
    return selectk_features, selected_feature_names

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)    
    return X_train, X_test, y_train, y_test,sc

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, cm

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(algorithm= 'auto', metric= 'minkowski', weights='uniform')
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def selectk_Classification(acknn):
    dataframe = pd.DataFrame(index=['Accuracy'], columns=['KNN'])
    dataframe['KNN'] = acknn
    return dataframe

In [2]:
dataset1 = pd.read_csv("balanced_dataset.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True)

indep_X = df2.drop('downtime', axis=1)
dep_Y = df2['downtime']
# k value = 5
kbest_features, selected_feature_names = selectkbest(indep_X, dep_Y, 5)

In [3]:
X_train, X_test, y_train, y_test,sc = split_scalar(pd.DataFrame(kbest_features), dep_Y)

accknn = []

In [4]:

classifier, Accuracy, report, cm = knn(X_train, y_train, X_test, y_test)  
accknn.append(Accuracy)

result = selectk_Classification(accknn)
print("Selected feature names:", selected_feature_names)      
print(result)


Selected feature names: ['response_time' 'error_rate' 'cpu_usage' 'memory_usage' 'active_users']
               KNN
Accuracy  0.815068


In [5]:
import pickle

In [6]:
filename="finalized_model_knn.pkl"

In [7]:
filename1="finalized_model_sc.pkl"

In [8]:
pickle.dump(classifier,open(filename,"wb"))

In [9]:
pickle.dump(sc,open(filename1,"wb"))

In [10]:
loaded_model=pickle.load(open("finalized_model_knn.pkl","rb"))

In [11]:
loaded_model1=pickle.load(open("finalized_model_sc.pkl","rb"))

In [12]:
preinput=loaded_model1.transform([[
    int(input("Enter the value of response_time:")),
    int(input("Enter the value of error_rate:")),
    int(input("Enter the value of cpu_usage:")),
    int(input("Enter the value of memory_usage:")),
    int(input("Enter the value of active_users:"))
    ]])
result=loaded_model.predict(preinput)
if (result==0):
    print("The application is not down for next 3 months")
else:
    print("The application is down for next 3 months")

Enter the value of response_time: 10
Enter the value of error_rate: 200
Enter the value of cpu_usage: 30
Enter the value of memory_usage: 40
Enter the value of active_users: 2


The application is down for next 3 months
